In [1]:
import numpy as np
import imutils
import cv2
import time

In [ ]:
prototxt = "MobileNetSSD_deploy.prototxt.txt"
model = "MobileNetSSD_deploy.caffemodel"
conf_thresh = 0.2

CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair",
           "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

print("Loading model...")
net = cv2.dnn.readNetFromCaffe(prototxt, model)
print("Model Loaded")
print("Starting Camera Feed...")

vs = cv2.VideoCapture(0)
time.sleep(2.0)

while True:
    _, frame = vs.read()
    frame = imutils.resize(frame, width=500)
    (h, w) = frame.shape[:2]
    im_resize_blob = cv2.resize(frame, (300, 300))
    blob = cv2.dnn.blobFromImage(im_resize_blob, 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()
    det_shape = detections.shape[2]

    # Flag to check if a bottle is detected
    bottle_detected = False

    for i in np.arange(0, det_shape):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_thresh:
            idx = int(detections[0, 0, i, 1])
            if CLASSES[idx] == "bottle":
                bottle_detected = True
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (start_x, start_y, end_x, end_y) = box.astype("int")
                label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
                cv2.rectangle(frame, (start_x, start_y), (end_x, end_y), COLORS[idx], 2)
                y = start_y - 15 if start_y - 15 > 15 else start_y + 15
                cv2.putText(frame, label, (start_x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

    # Print text if a bottle is detected
    if bottle_detected:
        cv2.putText(frame, "I need Water", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

vs.release()
cv2.destroyAllWindows()


Loading model...
Model Loaded
Starting Camera Feed...
